In [ ]:
#Total pages: 161 2021.7.14

In [ ]:
#Set up
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
!pip install beautifulsoup4

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [62.2 kB]
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1

In [ ]:
list_rows = []

In [ ]:
#Define getting news function
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select

options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--ignore-certificate-errors')
options.add_argument('--headless')
options.add_argument('--disable-dev-shm-usage')


temp_link = None
for pn in range(1,162):
  print("Getting Page "+str(pn))
  driver = webdriver.Chrome('chromedriver',options=options)

  wait = WebDriverWait(driver, 10)
  driver.get("https://www.politifact.com/factchecks/list/?page="+str(pn)+"&ruling=false")

  
  result = driver.find_element_by_class_name("o-listicle__list")

  for items in result.find_elements_by_class_name('o-listicle__item'):
   
      where = items.find_elements_by_class_name('m-statement__name')[0].text
      when = items.find_elements_by_class_name('m-statement__desc')[0].text
      title = items.find_elements_by_class_name('m-statement__quote')[0].text
      link = items.find_elements_by_class_name('m-statement__quote')[0].find_elements_by_tag_name('a')[0].get_attribute("href")
      list_cells = [where,when,title,link]
      list_rows.append(list_cells)

  driver.quit()

In [ ]:
list_rows

[['Mark Robinson',
  'stated on July 9, 2021 in a Facebook post:',
  'Says “showing up at the front doors of people’s houses and violating HIPAA laws by requesting private medical information regarding their vaccination status is unacceptable and illegal.”',
  'https://www.politifact.com/factchecks/2021/jul/15/mark-robinson/north-carolina-lt-governor-wrong-about-hipaa/'],
 ['Facebook posts',
  'stated on July 7, 2021 in Facebook:',
  '“BREAKING: Pennsylvania initiated a FULL audit of the RIGGED ELECTION”',
  'https://www.politifact.com/factchecks/2021/jul/14/facebook-posts/social-media-post-falsely-states-pennsylvania-has-/'],
 ['Facebook posts',
  'stated on July 11, 2021 in a Facebook post:',
  'A bodyguard of a high-ranking Democrat in Congress fired the shot that killed Ashli Babbitt on Jan. 6.',
  'https://www.politifact.com/factchecks/2021/jul/14/facebook-posts/officer-who-shot-ashli-babbitt-wasnt-assigned-any-/'],
 ['Donald Trump',
  'stated on July 11, 2021 in in an interview o

In [ ]:
#Creat datafram
import pandas as pd

df = pd.DataFrame(list_rows, columns = ['where','when','title','link'])

df

,where,when,title,link
0,Mark Robinson,"stated on July 9, 2021 in a Facebook post:",Says “showing up at the front doors of people’...,https://www.politifact.com/factchecks/2021/jul...
1,Facebook posts,"stated on July 7, 2021 in Facebook:",“BREAKING: Pennsylvania initiated a FULL audit...,https://www.politifact.com/factchecks/2021/jul...
2,Facebook posts,"stated on July 11, 2021 in a Facebook post:",A bodyguard of a high-ranking Democrat in Cong...,https://www.politifact.com/factchecks/2021/jul...
3,Donald Trump,"stated on July 11, 2021 in in an interview on ...",“There were no guns whatsoever” at the Capitol...,https://www.politifact.com/factchecks/2021/jul...
4,Viral image,"stated on July 8, 2021 in a Facebook post:",Says CNN reported that Joe Biden authorized “a...,https://www.politifact.com/factchecks/2021/jul...
...,...,...,...,...
4778,Mitt Romney,"stated on September 5, 2007 in a debate in Dur...","""The Z-visa that was offered in that Senate bi...",https://www.politifact.com/factchecks/2007/sep...
4779,Barack Obama,"stated on August 20, 2007 in New Hampshire:",If African-Americans vote their percentage of ...,https://www.politifact.com/factchecks/2007/sep...
4780,Rudy Giuliani,"stated on May 31, 2007 in New York.:","""The crime decline in the United States would ...",https://www.politifact.com/factchecks/2007/sep...
4781,Mike Huckabee,"stated on June 5, 2007 in Manchester, N.H.:","""I'll tell you what I can tell this country: I...",https://www.politifact.com/factchecks/2007/aug...


In [ ]:
import re
month = []
date = []
year = []
for idx,data in df.iterrows():
  # data = i[1]
  temp = data[1].split(',')

  date_time = temp[0][10:].split(' ')
  datetime_object = datetime.datetime.strptime(date_time[0], "%B")
  month.append(str(datetime_object.month))
  date.append(date_time[1])
  year.append(temp[1][1:5])

df['year'] = year
df['month'] = month
df['date'] = date

In [ ]:
df_selected = df[(df['year']>'2019')]
df_selected = df_selected.drop(df_selected[(df_selected['year'] == '2021') & (df_selected['month'] >'6')].index)
df_selected = df_selected.drop(df_selected[(df_selected['year'] == '2020') & (df_selected['month'] <'7')].index)
df_selected

,where,when,title,link,year,date,month
10,Instagram posts,"stated on June 29, 2021 in an Instagram post:",Sudden Infant Death Syndrome “is absolutely a ...,https://www.politifact.com/factchecks/2021/jul...,2021,29,6
13,Republican National Committee,"stated on June 29, 2021 in a tweet:","""Crime is escalating to a level we haven’t see...",https://www.politifact.com/factchecks/2021/jul...,2021,29,6
14,Facebook posts,"stated on June 30, 2021 in a Facebook post:",News organizations and sponsors ignored the ac...,https://www.politifact.com/factchecks/2021/jul...,2021,30,6
18,Just the News,"stated on June 17, 2021 in an article:","""Records suggest more than 100 batches of abse...",https://www.politifact.com/factchecks/2021/jul...,2021,17,6
19,Tucker Carlson,"stated on June 28, 2021 in a show:","“In Texas, the power companies have automatica...",https://www.politifact.com/factchecks/2021/jul...,2021,28,6
...,...,...,...,...,...,...,...
977,Brian Kilmeade,"stated on July 2, 2020 in an episode of ""Fox &...",The President’s Daily Brief “is like a mini no...,https://www.politifact.com/factchecks/2020/jul...,2020,2,7
979,Donald Trump,"stated on July 4, 2020 in a Fourth of July spe...",Says 99% of COVID-19 cases “are totally harmle...,https://www.politifact.com/factchecks/2020/jul...,2020,4,7
980,Facebook posts,"stated on July 2, 2020 in a Facebook post:",“People are starting to enter ERs with fungal ...,https://www.politifact.com/factchecks/2020/jul...,2020,2,7
981,Facebook posts,"stated on July 5, 2020 in a Facebook post:",Says Melania Trump’s dress at Mount Rushmore s...,https://www.politifact.com/factchecks/2020/jul...,2020,5,7


In [ ]:
covid_news = []

In [ ]:
for index,row in df.iterrows():

  if 'Covid' in row['title']:
      print("yes")
      covid_news.append(row)
  else:
      print("no")
covid_news

no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
yes
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no


[where                                      Instagram posts
 when         stated on June 28, 2021 in an Instagram post:
 title    Jill Biden implied the COVID-19 pandemic was p...
 link     https://www.politifact.com/factchecks/2021/jul...
 year                                                  2021
 date                                                    28
 month                                                    6
 Name: 5, dtype: object,
 where                                      Instagram posts
 when         stated on June 28, 2021 in an Instagram post:
 title    “CDC officials admit more hospitalizations of ...
 link     https://www.politifact.com/factchecks/2021/jun...
 year                                                  2021
 date                                                    28
 month                                                    6
 Name: 13, dtype: object,
 where                                      Instagram posts
 when         stated on June 23, 2021 in an Insta

In [ ]:
len(covid_news)

74

In [ ]:
import pandas as pd

df_cov = pd.DataFrame(covid_news, columns = ['where','when','title','link'])

df_cov

,where,when,title,link
5,Instagram posts,"stated on June 28, 2021 in an Instagram post:",Jill Biden implied the COVID-19 pandemic was p...,https://www.politifact.com/factchecks/2021/jul...
13,Instagram posts,"stated on June 28, 2021 in an Instagram post:",“CDC officials admit more hospitalizations of ...,https://www.politifact.com/factchecks/2021/jun...
15,Instagram posts,"stated on June 23, 2021 in an Instagram post:",Funeral directors say “no one is dying from CO...,https://www.politifact.com/factchecks/2021/jun...
20,Bloggers,"stated on June 14, 2021 in a story:",Says the U.S. government caused the coronaviru...,https://www.politifact.com/factchecks/2021/jun...
23,Instagram posts,"stated on June 21, 2021 in an Instagram post:",mRNA technology in COVID-19 vaccines was never...,https://www.politifact.com/factchecks/2021/jun...
...,...,...,...,...
687,Facebook posts,"stated on July 9, 2020 in a Facebook post:",“The CDC may have to stop calling COVID-19 an ...,https://www.politifact.com/factchecks/2020/jul...
689,Melissa Melendez,"stated on July 9, 2020 in a tweet:",“There is no science or data to suggest that k...,https://www.politifact.com/factchecks/2020/jul...
696,Donald Trump,"stated on July 4, 2020 in a Fourth of July spe...",Says 99% of COVID-19 cases “are totally harmle...,https://www.politifact.com/factchecks/2020/jul...
27,Facebook posts,"stated on June 22, 2021 in a social media post:",“The WORLD HEALTH ORGANIZATION recently revers...,https://www.politifact.com/factchecks/2021/jun...


In [ ]:
from google.colab import files
df_cov.to_csv('PolitiFact.csv', index=False)